In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
# GENERAL CONSTANTS & WINDOWS
TREND_WINDOW = 6  # hours for trend analysis
SUSTAINED_WINDOW_MINUTES = 120  # 2 hours for sustained hypotension check
SUSTAINED_PERCENTAGE = 0.7  # 70% of readings must meet criteria for persistence

# UNIVERSAL THRESHOLDS
THRESH_SPO2_LOW = 90  # Sepsis-related hypoxemia threshold
THRESH_SPO2_CRITICAL = 85  # Critical hypoxemia
THRESH_SHOCK_INDEX_HIGH = 0.9  # Shock index threshold for septic shock
THRESH_SHOCK_INDEX_SEVERE = 1.2  # Severe shock index
THRESH_TEMP_HIGH = 38.0  # Fever threshold
THRESH_TEMP_LOW = 36.0  # Hypothermia threshold

# AGE CATEGORY DEFINITIONS
AGE_THRESHOLDS = {
    'neonate': {
        'rr_low': 30,
        'rr_normal': 40,
        'rr_high': 60,
        'hr_low': 100,
        'hr_normal': 140,
        'hr_high': 160,
        'sbp_low': 60,
        'sbp_normal': 70,
        'sbp_high': 90,
        'temp_low': 36.0,
        'temp_normal': 37.0,
        'temp_high': 38.0
    },
    'infant': {
        'rr_low': 24,
        'rr_normal': 30,
        'rr_high': 40,
        'hr_low': 80,
        'hr_normal': 120,
        'hr_high': 140,
        'sbp_low': 70,
        'sbp_normal': 85,
        'sbp_high': 100,
        'temp_low': 36.0,
        'temp_normal': 37.0,
        'temp_high': 38.0
    },
    'child': {
        'rr_low': 18,
        'rr_normal': 22,
        'rr_high': 30,
        'hr_low': 70,
        'hr_normal': 90,
        'hr_high': 110,
        'sbp_low': 80,
        'sbp_normal': 95,
        'sbp_high': 110,
        'temp_low': 36.0,
        'temp_normal': 37.0,
        'temp_high': 38.0
    },
    'adolescent': {
        'rr_low': 12,
        'rr_normal': 16,
        'rr_high': 22,
        'hr_low': 60,
        'hr_normal': 80,
        'hr_high': 100,
        'sbp_low': 90,
        'sbp_normal': 105,
        'sbp_high': 120,
        'temp_low': 36.0,
        'temp_normal': 37.0,
        'temp_high': 38.0
    },
    'adult': {
        'rr_low': 12,
        'rr_normal': 16,
        'rr_high': 22,
        'hr_low': 60,
        'hr_normal': 80,
        'hr_high': 100,
        'sbp_low': 90,
        'sbp_normal': 115,
        'sbp_high': 130,
        'temp_low': 36.0,
        'temp_normal': 37.0,
        'temp_high': 38.0
    },
    'geriatric': {
        'rr_low': 12,
        'rr_normal': 16,
        'rr_high': 24,  # Slightly higher threshold for elderly
        'hr_low': 60,
        'hr_normal': 80,
        'hr_high': 110,  # Slightly higher threshold for elderly
        'sbp_low': 90,
        'sbp_normal': 120,
        'sbp_high': 140,  # Slightly higher threshold for elderly
        'temp_low': 35.5,  # Lower threshold for elderly (may have lower baseline)
        'temp_normal': 36.5,
        'temp_high': 37.8  # Lower fever threshold for elderly
    }
}

# SEPSIS-SPECIFIC THRESHOLDS
SEPSIS_THRESHOLDS = {
    'temp_high': 38.0,  # Fever threshold for sepsis
    'temp_low': 36.0,   # Hypothermia threshold for sepsis
    'rr_high': 22,      # Tachypnea threshold for sepsis
    'hr_high': 100,     # Tachycardia threshold for sepsis
    'sbp_low': 100,     # Hypotension threshold for sepsis
    'shock_index': 0.7, # Shock index threshold for sepsis
}

# SEPTIC SHOCK SPECIFIC THRESHOLDS
SEPTIC_SHOCK_THRESHOLDS = {
    'sbp_critical': 90,         # Critical hypotension for shock
    'shock_index_critical': 0.9, # Critical shock index for shock
    'lactate_elevated': 2.0,    # mmol/L - elevated lactate
    'lactate_severe': 4.0,      # mmol/L - severe hyperlactatemia
}

# ORGAN DYSFUNCTION THRESHOLDS (qSOFA components)
ORGAN_DYSFUNCTION_THRESHOLDS = {
    'gcs_low': 13,              # Glasgow Coma Scale ≤ 13
    'creatinine_high': 2.0,     # mg/dL - acute kidney injury
    'bilirubin_high': 2.0,      # mg/dL - liver dysfunction
    'platelets_low': 100,       # ×10^9/L - thrombocytopenia
    'inr_high': 1.5,            # Coagulopathy
}

# SCORING WEIGHTS FOR SEPSIS RISK ASSESSMENT
SEPSIS_SCORE_WEIGHTS = {
    'TEMP_ABNORMALITY': 1,          # Temp ≥ 38°C or ≤ 36°C
    'TACHYPNEA': 1,                 # RR ≥ 22
    'TACHYCARDIA': 1,               # HR ≥ 100
    'HYPOTENSION': 2,               # SBP ≤ 100
    'SHOCK_INDEX_ELEVATED': 2,      # SI ≥ 0.7
    'HYPOXEMIA': 2,                 # SpO2 < 90%
    'ALTERED_MENTAL_STATUS': 3,     # GCS ≤ 13
    'LACTATE_ELEVATED': 3,          # Lactate ≥ 2.0 mmol/L
    'PERSISTENT_HYPOTENSION': 4,    # SBP < 90 sustained
    'SHOCK_INDEX_CRITICAL': 4,      # SI ≥ 0.9
    'LACTATE_SEVERE': 4,            # Lactate ≥ 4.0 mmol/L
}

# SEPSIS RISK THRESHOLDS
SEPSIS_RISK_THRESHOLDS = {
    'LOW_RISK': 3,          # Score 0-3: Monitor
    'MODERATE_RISK': 6,     # Score 4-6: Possible sepsis, clinical review
    'HIGH_RISK': 9,         # Score 7-9: Probable sepsis, urgent intervention
    'SEVERE_RISK': 10,      # Score 10+: Septic shock likely, emergency treatment
}

# TREND ANALYSIS PARAMETERS
TREND_PARAMETERS = {
    'significant_rr_increase': 5,   # breaths/min increase per hour
    'significant_hr_increase': 10,  # beats/min increase per hour
    'significant_sbp_decrease': 10, # mmHg decrease per hour
    'significant_temp_increase': 0.5, # °C increase per hour
    'worsening_window_hours': 2,    # Time window for trend analysis
}

# PEDIATRIC SEPSIS CRITERIA (Additional thresholds for children)
PEDIATRIC_SEPSIS_THRESHOLDS = {
    'neonate_hr_high': 180,
    'infant_hr_high': 160, 
    'child_hr_high': 140,
    'neonate_rr_high': 60,
    'infant_rr_high': 50,
    'child_rr_high': 40,
    'cap_refill_delayed': 3,        # seconds - delayed capillary refill
    'urine_output_low': 1,          # mL/kg/hr - low urine output
}

In [3]:
def assign_age_category(df, age_col='age'):
    """Assign age category based on patient age with more granular categories"""
    df = df.copy()
    
    if age_col not in df.columns:
        # Default to adult if age not provided
        df['age_category'] = 'adult'
    else:
        def _categorize_age(age):
            if age <= 0.083: return 'neonate'     # < 1 month
            elif age <= 1: return 'infant'        # 1 month - 1 year
            elif age <= 5: return 'child'         # 1-5 years
            elif age <= 12: return 'child'        # 5-12 years
            elif age <= 18: return 'adolescent'   # 12-18 years
            elif age <= 65: return 'adult'        # 18-65 years
            else: return 'geriatric'              # > 65 years
        
        df['age_category'] = df[age_col].apply(_categorize_age)
    
    return df

In [4]:
def apply_vital_range_flags(df):
    df = df.copy()
    df = assign_age_category(df)  # Inject age group

    # SpO₂ flags
    df['flag_spo2_low'] = df['spo2'] < THRESH_SPO2_LOW

    # Temperature flags (both high & low for sepsis detection)
    df['flag_temp_high_sepsis'] = df['temperature'] >= SEPSIS_THRESHOLDS['temp_high']
    df['flag_temp_low_sepsis'] = df['temperature'] < SEPSIS_THRESHOLDS['temp_low']
    
    # Age-specific temperature flags (for general monitoring)
    df['flag_temp_high'] = df.apply(
        lambda row: row['temperature'] >= AGE_THRESHOLDS[row['age_category']]['temp_high'], axis=1)
    df['flag_temp_low'] = df.apply(
        lambda row: row['temperature'] < AGE_THRESHOLDS[row['age_category']]['temp_low'], axis=1)

    # RR flags - both age-specific and sepsis-specific
    df['flag_rr_high_sepsis'] = df['resp_rate'] >= SEPSIS_THRESHOLDS['rr_high']
    df['flag_rr_high'] = df.apply(
        lambda row: row['resp_rate'] >= AGE_THRESHOLDS[row['age_category']]['rr_high'], axis=1)

    # HR flags - both age-specific and sepsis-specific
    df['flag_hr_high_sepsis'] = df['heart_rate'] >= SEPSIS_THRESHOLDS['hr_high']
    df['flag_hr_high'] = df.apply(
        lambda row: row['heart_rate'] >= AGE_THRESHOLDS[row['age_category']]['hr_high'], axis=1)

    # SBP flags - both age-specific and sepsis-specific
    df['flag_sbp_low_sepsis'] = df['sbp'] <= SEPSIS_THRESHOLDS['sbp_low']
    df['flag_sbp_low'] = df.apply(
        lambda row: row['sbp'] < AGE_THRESHOLDS[row['age_category']]['sbp_low'], axis=1)
    
    # Shock index calculation and flag
    df['shock_index'] = np.where(df['sbp'] > 0, df['heart_rate'] / df['sbp'], np.nan)
    df['flag_si_high_sepsis'] = df['shock_index'] >= SEPSIS_THRESHOLDS['shock_index']
    df['flag_si_high_shock'] = df['shock_index'] >= SEPTIC_SHOCK_THRESHOLDS['shock_index_critical']

    return df

In [5]:
def compute_recent_trends_delta(df, sepsis_window=2):
    """
    Computes trends for each vital by differencing consecutive readings.
    Applies interpretation using both age-specific and sepsis-specific thresholds.
    """
    df = df.copy().sort_values("timestamp").reset_index(drop=True)

    if 'age_category' not in df.columns:
        df = assign_age_category(df)

    trends = {}
    recent = df.tail(TREND_WINDOW)
    sepsis_recent = df.tail(sepsis_window)  # Shorter window for sepsis trends
    
    if len(recent) == 0:
        return trends
        
    age_group = recent['age_category'].iloc[-1]
    thresholds = AGE_THRESHOLDS[age_group]

    vital_map = {
        'resp_rate': ('rr_low', 'rr_normal', 'rr_high'),
        'heart_rate': ('hr_low', 'hr_normal', 'hr_high'),
        'sbp': ('sbp_low', 'sbp_normal', 'sbp_high'),
        'temperature': ('temp_low', 'temp_normal', 'temp_high'),
        'spo2': (None, None, None)  # handled separately
    }

    # Standard trend analysis (using your original code)
    for vital in ['resp_rate', 'heart_rate', 'sbp', 'temperature', 'spo2']:
        if vital not in recent.columns or recent[vital].isnull().all():
            continue

        y = recent[vital].dropna().values
        if len(y) < 2:
            continue

        avg_delta = np.mean(np.diff(y))
        latest = y[-1]
        trends[f"{vital}_trend"] = round(avg_delta, 3)

        if vital == 'spo2':
            if latest < THRESH_SPO2_LOW:
                if avg_delta > 0:
                    flag = "Still abnormal — but improving"
                elif avg_delta < 0:
                    flag = "Abnormal and worsening"
                else:
                    flag = "Abnormal and flat"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"

        else:
            low_key, norm_key, high_key = vital_map[vital]
            low = thresholds[low_key]
            normal = thresholds[norm_key]
            high = thresholds[high_key]

            if latest < low or latest > high:
                if (latest > high and avg_delta < 0) or (latest < low and avg_delta > 0):
                    flag = "Still abnormal — but improving"
                else:
                    flag = "Abnormal and worsening"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"

        trends[f"{vital}_trend_flag"] = flag

    # Sepsis-specific trend analysis (NEW)
    for vital in ['resp_rate', 'heart_rate', 'sbp', 'temperature']:
        if vital not in sepsis_recent.columns or sepsis_recent[vital].isnull().all():
            continue
            
        y = sepsis_recent[vital].dropna().values
        if len(y) < 2:
            continue
            
        # Calculate rapid change (per hour)
        time_diff_hours = (sepsis_recent['timestamp'].iloc[-1] - sepsis_recent['timestamp'].iloc[0]).total_seconds() / 3600
        if time_diff_hours == 0:
            continue
            
        total_change = y[-1] - y[0]
        change_per_hour = total_change / time_diff_hours
        
        # Check against sepsis trend thresholds
        if vital == 'resp_rate' and change_per_hour >= TREND_PARAMETERS['significant_rr_increase']:
            trends['sepsis_rr_trend'] = f"Rapid increase: {change_per_hour:.1f}/hour"
        elif vital == 'heart_rate' and change_per_hour >= TREND_PARAMETERS['significant_hr_increase']:
            trends['sepsis_hr_trend'] = f"Rapid increase: {change_per_hour:.1f}/hour"
        elif vital == 'sbp' and change_per_hour <= -TREND_PARAMETERS['significant_sbp_decrease']:
            trends['sepsis_sbp_trend'] = f"Rapid decrease: {change_per_hour:.1f}/hour"
        elif vital == 'temperature' and change_per_hour >= TREND_PARAMETERS['significant_temp_increase']:
            trends['sepsis_temp_trend'] = f"Rapid increase: {change_per_hour:.1f}/hour"

    # Shock Index trend (enhanced)
    if all(col in recent.columns for col in ['heart_rate', 'sbp']):
        hr = recent['heart_rate'].values
        sbp = np.clip(recent['sbp'].values, a_min=1, a_max=None)
        si = hr / sbp

        if len(si) >= 2:
            avg_si_delta = np.mean(np.diff(si))
            trends['shock_index_trend'] = round(avg_si_delta, 3)

            latest_si = si[-1]
            if latest_si >= SEPTIC_SHOCK_THRESHOLDS['shock_index_critical']:
                flag = "Critical Shock Index — improving" if avg_si_delta < 0 else "Critical Shock Index — worsening"
            elif latest_si >= SEPSIS_THRESHOLDS['shock_index']:
                flag = "Elevated Shock Index — improving" if avg_si_delta < 0 else "Elevated Shock Index — worsening"
            else:
                flag = "Normal but improving" if avg_si_delta < 0 else "Normal but rising"

            trends['shock_index_trend_flag'] = flag

    return trends

In [6]:
def detect_sepsis(df):
    """
    Detect sepsis based on SIRS criteria (≥2 of the following):
    1. Temp ≥ 38°C or ≤ 36°C
    2. RR ≥ 22
    3. HR ≥ 100  
    4. SBP ≤ 100
    Uses the sepsis-specific flags from apply_vital_range_flags
    """
    df = df.copy()
    
    # Count SIRS criteria using sepsis-specific flags
    sirs_criteria_count = (
        (df['flag_temp_high_sepsis'] | df['flag_temp_low_sepsis']).astype(int) +  # Temperature abnormality
        df['flag_rr_high_sepsis'].astype(int) +                                   # RR ≥ 22
        df['flag_hr_high_sepsis'].astype(int) +                                   # HR ≥ 100  
        df['flag_sbp_low_sepsis'].astype(int)                                     # SBP ≤ 100
    )
    
    df['flag_sepsis'] = sirs_criteria_count >= 2
    return df

def detect_septic_shock(df, persistence_window=3):
    """
    Detect septic shock based on sepsis + persistent hypotension
    (SBP < 90 or SI ≥ 0.9 for consecutive readings)
    """
    df = df.copy()
    
    # Check hypotension criteria for septic shock
    hypotension_criteria = (df['sbp'] < SEPTIC_SHOCK_THRESHOLDS['sbp_critical']) | (df['shock_index'] >= SEPTIC_SHOCK_THRESHOLDS['shock_index_critical'])
    
    # Check for persistent hypotension (using rolling window)
    # We need at least 'persistence_window' consecutive True values
    df['persistent_hypotension'] = hypotension_criteria.rolling(
        window=persistence_window, min_periods=persistence_window
    ).apply(lambda x: all(x), raw=True)
    
    # Septic shock requires both sepsis and persistent hypotension
    df['flag_septic_shock'] = df['flag_sepsis'] & df['persistent_hypotension']
    
    return df

def evaluate_sepsis_risk(df, trends):
    """
    Calculate a sepsis risk score based on flags and trends
    """
    risk_score = 0
    
    # Basic SIRS criteria
    if df['flag_temp_high_sepsis'].iloc[-1] or df['flag_temp_low_sepsis'].iloc[-1]:
        risk_score += SEPSIS_SCORE_WEIGHTS['TEMP_ABNORMALITY']
    if df['flag_rr_high_sepsis'].iloc[-1]:
        risk_score += SEPSIS_SCORE_WEIGHTS['TACHYPNEA']
    if df['flag_hr_high_sepsis'].iloc[-1]:
        risk_score += SEPSIS_SCORE_WEIGHTS['TACHYCARDIA']
    if df['flag_sbp_low_sepsis'].iloc[-1]:
        risk_score += SEPSIS_SCORE_WEIGHTS['HYPOTENSION']
    
    # Shock index
    if df['flag_si_high_sepsis'].iloc[-1]:
        risk_score += SEPSIS_SCORE_WEIGHTS['SHOCK_INDEX_ELEVATED']
    if df['flag_si_high_shock'].iloc[-1]:
        risk_score += SEPSIS_SCORE_WEIGHTS['SHOCK_INDEX_CRITICAL']
    
    # Hypoxemia
    if df['flag_spo2_low'].iloc[-1]:
        risk_score += SEPSIS_SCORE_WEIGHTS['HYPOXEMIA']
    
    # Persistent hypotension
    if df['persistent_hypotension'].iloc[-1]:
        risk_score += SEPSIS_SCORE_WEIGHTS['PERSISTENT_HYPOTENSION']
    
    # Trend-based scoring
    if 'sepsis_rr_trend' in trends:
        risk_score += 1  # Add point for rapid RR increase
    if 'sepsis_hr_trend' in trends:
        risk_score += 1  # Add point for rapid HR increase
    if 'sepsis_sbp_trend' in trends:
        risk_score += 2  # Add points for rapid SBP decrease
    if 'sepsis_temp_trend' in trends:
        risk_score += 1  # Add point for rapid temp increase
    
    # Determine risk level
    if risk_score >= SEPSIS_RISK_THRESHOLDS['SEVERE_RISK']:
        risk_level = 'SEVERE_RISK'
    elif risk_score >= SEPSIS_RISK_THRESHOLDS['HIGH_RISK']:
        risk_level = 'HIGH_RISK'
    elif risk_score >= SEPSIS_RISK_THRESHOLDS['MODERATE_RISK']:
        risk_level = 'MODERATE_RISK'
    else:
        risk_level = 'LOW_RISK'
    
    return risk_score, risk_level

# Updated pipeline integration
def evaluate_patient(self, assume_infection_risk=True):
    """Evaluate all patient data for sepsis and septic shock"""
    # Apply vital range flags first
    df_with_flags = apply_vital_range_flags(self.patient_data)
    
    # Detect sepsis
    df_with_sepsis = detect_sepsis(df_with_flags)
    
    # Detect septic shock
    df_with_shock = detect_septic_shock(df_with_sepsis)
    
    # Analyze trends
    trends = compute_recent_trends_delta(df_with_shock)
    
    # Calculate risk score
    risk_score, risk_level = evaluate_sepsis_risk(df_with_shock, trends)
    
    # Convert to results format
    results = []
    for _, row in df_with_shock.iterrows():
        result = {
            'timestamp': row['timestamp'],
            'temperature': row['temperature'],
            'resp_rate': row['resp_rate'],
            'sbp': row['sbp'],
            'heart_rate': row['heart_rate'],
            'spo2': row.get('spo2', None),
            'shock_index': row.get('shock_index', None),
            'sepsis_alert': row['flag_sepsis'],
            'septic_shock_alert': row['flag_septic_shock'],
            'risk_score': risk_score if _ == len(df_with_shock) - 1 else None,
            'risk_level': risk_level if _ == len(df_with_shock) - 1 else None,
            'flags': {
                'temp_high_sepsis': row['flag_temp_high_sepsis'],
                'temp_low_sepsis': row['flag_temp_low_sepsis'],
                'rr_high_sepsis': row['flag_rr_high_sepsis'],
                'hr_high_sepsis': row['flag_hr_high_sepsis'],
                'sbp_low_sepsis': row['flag_sbp_low_sepsis'],
                'si_high_sepsis': row['flag_si_high_sepsis'],
                'si_high_shock': row['flag_si_high_shock'],
                'spo2_low': row.get('flag_spo2_low', False),
            }
        }
        results.append(result)
    
    # Add trends to the latest result
    if results:
        results[-1]['trends'] = trends
    
    return results

In [7]:
if __name__ == "__main__":
    # Simple test data
    test_data = pd.DataFrame({
        'timestamp': pd.date_range(start='2023-10-27 10:00', periods=6, freq='30min'),
        'temperature': [37.0, 37.5, 38.2, 38.5, 38.3, 38.6],  # °C - developing fever
        'resp_rate': [18, 20, 22, 24, 25, 26],                # breaths/min - increasing
        'sbp': [115, 105, 98, 92, 88, 85],                    # mmHg - decreasing
        'heart_rate': [85, 92, 102, 110, 115, 118],           # beats/min - increasing
        'spo2': [96, 94, 92, 90, 89, 88],                     # % - decreasing
        'age': [45, 45, 45, 45, 45, 45]                       # years
    })
    
    # Test the functions
    flagged_data = apply_vital_range_flags(test_data)
    sepsis_data = detect_sepsis(flagged_data)
    shock_data = detect_septic_shock(sepsis_data)
    
    trends = compute_recent_trends_delta(shock_data)
    risk_score, risk_level = evaluate_sepsis_risk(shock_data, trends)
    print(f"Sepsis detected: {sepsis_data['flag_sepsis'].iloc[0]}")
    print(f"Septic shock detected: {shock_data['flag_septic_shock'].iloc[0]}")
    print(f"Shock index: {shock_data['shock_index'].iloc[0]:.2f}")
    print(f"Risk score: {risk_score}")
    print(f"Risk level: {risk_level}")

Sepsis detected: False
Septic shock detected: False
Shock index: 0.74
Risk score: 18
Risk level: SEVERE_RISK


In [8]:
if __name__ == "__main__":
    # Simple test data
    test_data = pd.DataFrame({
        'timestamp': pd.date_range(start='2023-10-27 10:00', periods=6, freq='30min'),
        'temperature': [37.0, 37.5, 38.2, 38.5, 38.3, 38.6],  # °C - developing fever
        'resp_rate': [18, 20, 22, 24, 25, 26],                # breaths/min - increasing
        'sbp': [115, 105, 98, 92, 88, 85],                    # mmHg - decreasing
        'heart_rate': [85, 92, 102, 110, 115, 118],           # beats/min - increasing
        'spo2': [96, 94, 92, 90, 89, 88],                     # % - decreasing
        'age': [45, 45, 45, 45, 45, 45]                       # years
    })
    
    # Test the functions
    flagged_data = apply_vital_range_flags(test_data)
    sepsis_data = detect_sepsis(flagged_data)
    shock_data = detect_septic_shock(sepsis_data)
    
    # ADD THIS LINE for trend analysis
    trends = compute_recent_trends_delta(shock_data)
    
    
    print(f"Sepsis detected: {sepsis_data['flag_sepsis'].iloc[-1]}")
    print(f"Septic shock detected: {shock_data['flag_septic_shock'].iloc[-1]}")
    print(f"Shock index: {shock_data['shock_index'].iloc[-1]:.2f}")
    
    # ADD THESE LINES to see trends and flags
    print("\nTREND ANALYSIS")
    for key, value in trends.items():
        if 'flag' in key or 'trend' in key:
            print(f"{key}: {value}")
    
    print("\nLATEST FLAGS")
    latest_flags = {k: v for k, v in shock_data.iloc[-1].items() if 'flag' in k}
    for flag, value in latest_flags.items():
        print(f"{flag}: {value}")

Sepsis detected: True
Septic shock detected: True
Shock index: 1.39

TREND ANALYSIS
resp_rate_trend: 1.6
resp_rate_trend_flag: Abnormal and worsening
heart_rate_trend: 6.6
heart_rate_trend_flag: Abnormal and worsening
sbp_trend: -6.0
sbp_trend_flag: Abnormal and worsening
temperature_trend: 0.32
temperature_trend_flag: Abnormal and worsening
spo2_trend: -1.6
spo2_trend_flag: Abnormal and worsening
sepsis_temp_trend: Rapid increase: 0.6/hour
shock_index_trend: 0.13
shock_index_trend_flag: Critical Shock Index — worsening

LATEST FLAGS
flag_spo2_low: True
flag_temp_high_sepsis: True
flag_temp_low_sepsis: False
flag_temp_high: True
flag_temp_low: False
flag_rr_high_sepsis: True
flag_rr_high: True
flag_hr_high_sepsis: True
flag_hr_high: True
flag_sbp_low_sepsis: True
flag_sbp_low: True
flag_si_high_sepsis: True
flag_si_high_shock: True
flag_sepsis: True
flag_septic_shock: True
